In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
try:
    from rich.progress import track
except:
    track = lambda x: x

from numpy.random import default_rng
rng = default_rng()

In [ ]:
class TwoDLapl:
    def __init__(self, N:int, **kwargs):
        self.V = np.empty((N,N), dtype=np.float64)
        self.N = N
        self.N1 = N-1
        self.N2 = N-2
        self.shift = []
        self.im = None
        self.reset()
        self.SOR = kwargs.get('SOR', 1.0)
        self.order = self.set_order(**kwargs)
        self.alternate = kwargs.get('alternate', False)
        self.fig, (self.axim, self.axline) = plt.subplots(
            ncols=2, figsize=(12,6),
            gridspec_kw=dict(width_ratios=(2,1)))
        self.im = self.axim.matshow(self.V, cmap='Spectral', origin='lower', extent=(0,1,0,1))
        plt.colorbar(self.im, shrink=0.75)

    def reset(self):
        self.V[:,:] = 0.0
        self.V[self.N1,:] = 1.0
        self.shift = []

    def relax(self):
        self.dVmax = 0.0
        V = self.V

        def update(r, c):
            newV = (V[r+1,c] + V[r-1,c] + V[r,c+1] + V[r,c-1]) / 4
            dV = newV - V[r,c]
            self.dVmax = max(self.dVmax, abs(dV))
            V[r,c] += self.SOR * dV

        if self.alternate:
            o = self.order if len(self.shift) % 2 else reversed(self.order)
        else:
            o = self.order

        for r,c in o:
            update(r, c)

        self.shift.append(self.dVmax)
    
    def set_order(self, **kwargs):
        """Create a list of the (r,c) pairs to update"""
        interior = np.arange(1, self.N1, dtype=np.uint32)
        X, Y = np.meshgrid(interior, interior)
        allpairs = [(x,y) for x,y in zip(X.flatten(), Y.flatten())]
        
        if kwargs.get('random', False):
            allpairs = np.array(allpairs)
            rng.shuffle(allpairs)
        
        return tuple(allpairs)

    def show(self):
        self.im.set_data(self.V)
        self.axline.clear()
        self.axline.semilogy(np.arange(len(self.shift)), self.shift)
        self.axline.set_ylabel("abs(shift)")
        self.axline.set_xlabel("iteration")
        self.fig.canvas.draw()
        self.fig.canvas.flush_events()

    def showrelax(self, N:int, update:int):
        for n in track(range(N)):
            self.relax()
            if (n+1) % update == 0:
                self.show()

In [ ]:
d.showrelax(1000,50)

In [ ]:
dd = TwoDLapl(51, random=True)

In [ ]:
dd.showrelax(1000,100)

In [ ]:
dSOR = TwoDLapl(101, SOR=1.8)

In [ ]:
dSOR.showrelax(1000,100)

In [ ]:
dbig = TwoDLapl(201, SOR=1.8)

In [ ]:
dbig.showrelax(1000,100)